In [1]:
import numpy as np
import pandas as pd
import itertools
import larch
import os
import re
import yaml

from pytest import approx
from itertools import combinations

# Estimating Coordinated Daily Activity Patterns

This notebook illustrates how to re-estimate ActivitySim's CDAP model.  The steps in the process are:
  - Run ActivitySim in estimation mode to read household travel survey files, run the ActivitySim submodels to write estimation data bundles (EDB) that contains the model utility specifications, coefficients, chooser data, and alternatives data for each submodel.
  - Read and transform the relevant EDB into the format required by the model estimation package [larch](https://larch.newman.me) and then re-estimate the model coefficients.  No changes to the model specification will be made.
  - TODO: Update the ActivitySim model coefficients and re-run the model in simulation mode.
  

In [2]:
from larch.util.activitysim import cdap_model

In [3]:
#!activitysim create -e example_estimation_sf -d test

In [4]:
%cd test

/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/activitysim/activitysim/examples/example_estimation/notebooks/test


In [5]:
#!activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

In [6]:
edb_directory = "output/estimation_data_bundle/cdap/"
os.path.exists(edb_directory)

True

In [7]:
def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

def read_yaml(filename, **kwargs):
    with open(os.path.join(edb_directory, filename), 'rt') as f:
        return yaml.load(f, Loader=yaml.SafeLoader, **kwargs)

# Load Inputs

## Settings

In [8]:
settings = read_yaml("cdap_model_settings.yaml")
settings

{'INDIV_AND_HHSIZE1_SPEC': 'cdap_indiv_and_hhsize1.csv',
 'COEFFICIENTS': 'cdap_coefficients.csv',
 'FIXED_RELATIVE_PROPORTIONS_SPEC': 'cdap_fixed_relative_proportions.csv',
 'CONSTANTS': {'FULL': 1,
  'PART': 2,
  'UNIVERSITY': 3,
  'NONWORK': 4,
  'RETIRED': 5,
  'DRIVING': 6,
  'SCHOOL': 7,
  'PRESCHOOL': 8},
 'annotate_persons': {'SPEC': 'annotate_persons_cdap', 'DF': 'persons'},
 'annotate_households': {'SPEC': 'annotate_households_cdap',
  'DF': 'households',
  'TABLES': ['persons']}}

## Households

In [9]:
hhs = pd.read_csv("./data_sf/households.csv")
hhs

,household_id,TAZ,income,hhsize,HHT,auto_ownership,num_workers
0,2223759,16,144100,2,1,0,2
1,990869,134,48000,2,1,2,2
2,125886,113,25900,1,4,1,1
3,727893,8,26100,2,1,0,1
4,2741769,150,121600,4,1,2,1
...,...,...,...,...,...,...,...
1995,663493,110,19180,1,6,1,1
1996,569375,20,7400,1,6,1,0
1997,1445193,17,75000,1,4,0,1
1998,2833455,69,0,1,0,0,0


## Persons

In [10]:
persons = pd.read_csv("./data_sf/persons.csv")
persons

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype
0,166,166,54,1,2,3,3,4
1,197,197,46,1,2,3,3,4
2,268,268,46,1,1,3,3,4
3,375,375,54,1,2,3,3,4
4,387,387,44,1,2,3,3,4
...,...,...,...,...,...,...,...,...
4401,7554799,2863464,93,1,2,3,3,5
4402,7554818,2863483,68,1,1,3,3,5
4403,7555141,2863806,93,1,2,3,3,5
4404,7555853,2864518,71,1,1,3,3,5


In [11]:
from activitysim.abm.models.util.cdap import assign_cdap_rank
person_rank = assign_cdap_rank(persons)

## Coefficients

In [12]:
coefficients = read_csv(
    "cdap_coefficients.csv",
    index_col='coefficient_name',
    comment="#",
)
coefficients


,value,constrain
coefficient_name,,
coef_UNAVAILABLE,-999.000000,T
coef_full_time_worker_asc_M,1.378735,F
coef_full_time_worker_asc_N,0.622662,F
coef_part_time_worker_asc_M,-0.718824,F
coef_part_time_worker_asc_N,0.636032,F
...,...,...
coef_N_xxxx,-1.346000,F
coef_H_xxxx,-3.733000,F
coef_M_xxxxx,-1.528000,F


## Interaction Coefficients

In [13]:
interaction_coef = read_csv(
    "cdap_interaction_coefficients.csv",
    dtype={
        'interaction_ptypes':str,
    },
    keep_default_na=False,
    comment="#",
)
# interaction_coef['cardinality'] = interaction_coef['interaction_ptypes'].str.len()
interaction_coef

,activity,interaction_ptypes,coefficient
0,H,11,coef_H_11
1,H,12,coef_H_12
2,H,13,coef_H_13
3,H,14,coef_H_14
4,H,15,coef_H_15
...,...,...,...
128,N,****,coef_N_xxxx
129,H,****,coef_H_xxxx
130,M,*****,coef_M_xxxxx
131,N,*****,coef_N_xxxxx


## Spec 1

In [14]:
spec1 = read_csv("cdap_INDIV_AND_HHSIZE1_SPEC.csv", comment='#')
spec1.head()

,Description,Expression,M,N,H
0,Full-time worker alternative-specific constants,ptype == 1,coef_full_time_worker_asc_M,coef_full_time_worker_asc_N,NaN
1,Part-time worker alternative-specific constants,ptype == 2,coef_part_time_worker_asc_M,coef_part_time_worker_asc_N,NaN
2,University student alternative-specific constants,ptype == 3,coef_university_student_asc_M,coef_university_student_asc_N,NaN
3,Non-working adult alternative-specific constants,ptype == 4,coef_UNAVAILABLE,coef_non_working_adult_asc_N,NaN
4,Retired alternative-specific constants,ptype == 5,coef_UNAVAILABLE,coef_retired_asc_N,NaN


## Values

In [15]:
values = read_csv("cdap_values_combined.csv", comment='#')
values.head()

,person_id,model_choice,override_choice,household_id,age,PNUM,sex,pemploy,pstudent,ptype,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
0,166,N,N,166,54,1,2,3,3,4,...,0.0,2,0.0,0.00000,0.00000,1,3.21263,24.783133,31.566265,13.883217
1,197,N,N,197,46,1,2,3,3,4,...,116.0,2,0.0,0.00000,0.00000,1,3.68156,56.783784,10.459459,8.832526
2,268,N,N,268,46,1,1,3,3,4,...,0.0,1,0.0,3598.08521,0.00000,1,3.29100,11.947644,45.167539,9.448375
3,375,N,N,375,54,1,2,3,3,4,...,68.0,1,0.0,0.00000,0.00000,1,4.11499,73.040169,28.028350,20.255520
4,387,N,N,387,44,1,2,3,3,4,...,0.0,3,0.0,227.78223,41.22827,1,3.83527,26.631579,45.868421,16.848945


In [16]:
values['cdap_rank'] = persons['cdap_rank']

# Model

In [17]:
model = cdap_model(hhs, values, spec1, interaction_coef, coefficients)

one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>


In [18]:
model[0].utility_functions() ## HH Size = 1 

<xmle.Elem 'div' with 1 children>

In [19]:
model[1].utility_functions() ## HH Size = 2

<xmle.Elem 'div' with 1 children>

In [20]:
model.estimate(method='SLSQP', options={'maxiter':1000})

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_UNAVAILABLE,-999.000000,0.0,0.0,NaN,NaN,True,,-999.000000
coef_child_who_is_in_school_or_too_young_for_school_interaction_with_off_peak_accessibility_to_retail_N,0.814567,0.0,0.0,NaN,NaN,False,,0.814567
coef_driving_age_child_who_is_in_school_asc_M,0.731969,0.0,0.0,NaN,NaN,False,,0.731969
coef_driving_age_child_who_is_in_school_asc_N,-8.478587,0.0,0.0,NaN,NaN,False,,-8.478587
coef_driving_age_child_who_is_in_school_interaction_income_between_50k_and_100k_H,-0.899233,0.0,0.0,NaN,NaN,False,,-0.899233
...,...,...,...,...,...,...,...,...
coef_M_xxxx,-0.071483,0.0,0.0,NaN,NaN,False,,-0.071483
coef_N_xxxx,-0.676366,0.0,0.0,NaN,NaN,False,,-0.676366
coef_H_xxxxx,-27.301540,0.0,0.0,NaN,NaN,False,,-27.301540
coef_M_xxxxx,-0.304403,0.0,0.0,NaN,NaN,False,,-0.304403


<ipython-input-20-dbc5e2c47bd5>:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  model.estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.344880703578103e-32 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: coef_UNAVAILABLE                                                                                          -999.000000
┃                   coef_child_who_is_in_school_or_too_young_for_school_interaction_with_off_peak_accessibility_to_retail_N      0.814567
┃                   coef_driving_age_child_who_is_in_school_asc_M                                                                0.731969
┃                   coef_driving_age_child_who_is_in_school_asc_N                                                               -8.478587
┃                   coef_driving_age_child_who_is_in_school_interaction_income_between_50k_and_100k_H                           -0.899233
┃                                                                                                                                 ...    
┃                   coef_M_xxxx                                                                                                 -0.071483
┃                   coef_N_xxxx                                                                                                 -0.676366
┃                   coef_H_xxxxx                                                                                               -27.301540
┃                   coef_M_xxxxx                                                                                                -0.304403
┃                   coef_N_xxxxx                                                                                               -20.945379
┃                   Length: 166, dtype: float64
┣          loglike: -2332.0740809055474
┣        d_loglike: coef_UNAVAILABLE                                                                                           0.000000e+00
┃                   coef_child_who_is_in_school_or_too_young_for_school_interaction_with_off_peak_accessibility_to_retail_N   -3.508050e-04
┃                   coef_driving_age_child_who_is_in_school_asc_M                                                             -2.667242e-04
┃                   coef_driving_age_child_who_is_in_school_asc_N                                                             -4.401232e-05
┃                   coef_driving_age_child_who_is_in_school_interaction_income_between_50k_and_100k_H                          1.667903e-05
┃                                                                                                                                  ...     
┃                   coef_M_xxxx                                                                                                6.617791e-05
┃                   coef_N_xxxx                                                                                               -1.472613e-04
┃                   coef_H_xxxxx                                                                                              -5.455571e-10
┃                   coef_M_xxxxx                                                                                               3.673193e-06
┃                   coef_N_xxxxx                                                                                              -1.616384e-08
┃                   Length: 166, dtype: float64
┣              nit: 126
┣             nfev: 261
┣             njev: 126
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=80, microseconds=850491)
┣           method: 'SLSQP'
┣          n_cases: 2000
┣ iteration_number: 126
┣          logloss: 1.1660370404527738

In [21]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value
coef_UNAVAILABLE,-999.,0.00,NA,,NA,0.00
coef_child_who_is_in_school_or_too_young_for_school_interaction_with_off_peak_accessibility_to_retail_N,0.815,0.616,1.32,,NA,0.00
coef_driving_age_child_who_is_in_school_asc_M,0.732,0.655,1.12,,NA,0.00
coef_driving_age_child_who_is_in_school_asc_N,-8.48,6.15,-1.38,,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_income_between_50k_and_100k_H,-0.899,0.760,-1.18,,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_with_fewer_cars_than_workers_H,0.454,0.672,0.68,,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_with_income_more_than_100k_H,-1.77,0.890,-1.99,*,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_with_less_than_20k_H,0.458,0.810,0.57,,NA,0.00
coef_full_time_worker_asc_M,2.05,2.62,0.78,,NA,0.00
coef_full_time_worker_asc_N,0.761,0.192,3.97,***,NA,0.00


You should be careful re-estimating this model.  If may exhibit some instability 
if there is not enough data to support all the parameters being estimated.

In [22]:
model.set_values('null')
model_d_param = model.d_loglike()
model.set_values('best')
model_d_param[model_d_param.abs() < 0.0001]
# These parameters appear to have no effect on likelihood using the given data

coef_UNAVAILABLE                                                                  0.0
coef_non_working_adult_interaction_with_female_gender_M                           0.0
coef_non_working_adult_interaction_with_more_cars_than_workers_M                  0.0
coef_non_working_adult_interaction_with_peak_accessibility_to_all_employment_M    0.0
coef_retired_interaction_with_female_M                                            0.0
coef_retired_interaction_with_more_cars_than_workers_M                            0.0
coef_retired_interaction_with_peak_accessibility_to_all_employment_M              0.0
coef_H_36                                                                         0.0
coef_H_37                                                                         0.0
coef_H_38                                                                         0.0
coef_H_46                                                                         0.0
coef_H_47                                             

In [23]:
model.possible_overspecification
# These parameters might be overspecified

,(1) 1.076e-25,(2) 4.241e-16,(3) 9.076e-16,(4) 9.655e-16,(5) 3.657e-14,(6) 7.038e-12,(7) 5.208e-11,(8) 6.128e-11,(9) 1.089e-10,(10) 1.139e-10,(11) 1.749e-09,(12) 2.795e-09,(13) 3.5e-09,(14) 5.269e-09,(15) 5.62e-09,(16) 1.03e-08,(17) 2.608e-08,(18) 3.154e-07,(19) 1.345e-06
coef_N_246_226_446,-1.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
coef_H_25,nan,-0.000004,-0.000000,0.000001,-0.000003,0.000010,-1.000000,-0.000181,0.000011,0.000000,nan,nan,0.000000,nan,nan,0.000000,-0.000000,nan,nan
coef_H_28,nan,-0.000000,nan,-0.000000,-0.000000,0.000000,-0.000011,-0.000006,-1.000000,-0.000019,0.000000,-0.000000,-0.000000,0.000000,nan,-0.000001,0.000000,nan,nan
coef_H_67,nan,-0.000000,nan,-0.000000,-0.000000,-0.000006,-0.000181,1.000000,-0.000006,-0.000007,0.000000,nan,-0.000002,nan,nan,-0.000003,0.000000,nan,nan
coef_M_67,nan,-0.000006,0.000000,0.000017,0.000046,-1.000000,-0.000010,-0.000006,-0.000000,0.000000,nan,nan,0.000000,nan,nan,0.000000,nan,nan,nan
coef_M_78,nan,0.466786,0.045388,-0.883185,-0.005962,-0.000018,-0.000003,0.000000,nan,nan,0.000000,-0.000000,nan,nan,nan,nan,nan,nan,nan
coef_N_66,nan,0.006005,0.024991,-0.002290,0.999667,0.000045,-0.000003,0.000000,-0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
coef_N_67,nan,-0.000000,nan,nan,nan,nan,nan,nan,nan,0.003377,-0.000000,-0.999994,0.000084,-0.000000,-0.000000,0.000013,0.000001,nan,nan
coef_N_78,nan,0.884287,-0.036015,0.465540,-0.003345,0.000002,-0.000003,0.000000,-0.000000,nan,nan,-0.000000,nan,nan,nan,nan,nan,nan,nan
coef_N_166,nan,-0.010532,-0.998007,-0.057023,0.024882,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
